In [5]:
# Load packages
import IPython.display as ipd
import os
import json
import random
import pydub
import numpy as np
import torch
import copy
from torch.utils.data import DataLoader
from models import Wav2Letter
torch.cuda.set_device(1)
# Import 
from utils import AudioDataset, PostProcess
root_dir = '../../ssd/audio'

In [6]:
# Create a list of paths
kore_word_dir = os.path.join(root_dir,'kore_words')

from glob import glob
audio_dir = os.path.join(kore_word_dir,'word_audio_npy')
glob_pattern = os.path.join(audio_dir, '*')
audio_list = sorted(glob(glob_pattern), key=os.path.getctime)

target_dir = os.path.join(kore_word_dir,'targets')
glob_pattern = os.path.join(target_dir, '*')
target_list = sorted(glob(glob_pattern), key=os.path.getctime)

In [7]:
n_audio_max = 80000
n_target_max = 9
dataset = AudioDataset(audio_list,target_list,n_audio_max,n_target_max)
lookup_dict = json.load(open('./lookup.json'))
postprocessor = PostProcess(lookup_dict)
random_integer = random.randint(0,len(audio_list))
original_word_audio_dir =  os.path.join(kore_word_dir,'kore-sound-vocab-munged')

random_audio, random_target, _ = dataset[random_integer]
random_audio_ID = os.path.split(audio_list[random_integer])[-1][:-4]
original_audio_path = os.path.join(original_word_audio_dir,random_audio_ID) + '.mp3'

rs = int(pydub.utils.mediainfo(original_audio_path)['sample_rate'])

print(postprocessor.target2kana(random_target.cpu().numpy()))
ipd.display(ipd.Audio(random_audio.cpu().numpy()*2**15, rate=rs,autoplay=True))

キタカゼ


In [9]:
n_dataset = len(audio_list)
train_proportion = .9
n_train = int(.9*n_dataset)
inds = np.arange(n_dataset)
np.random.shuffle(inds)
inds_train = inds[:n_train]
inds_val = inds[n_train:]
n_val = n_dataset - n_train

audio_list_train = np.array(audio_list)[inds_train].tolist()
audio_list_val = np.array(audio_list)[inds_val].tolist()

target_list_train = np.array(target_list)[inds_train].tolist()
target_list_val = np.array(target_list)[inds_val].tolist()

In [10]:
# I know that it possible to use the Subset class to separate the train and validation loaders,
# however I don't know how to make the validation dataset not randomly subsampled in that case

dataset_train = AudioDataset(audio_list_train,target_list_train,n_audio_max,n_target_max)
dataset_val = AudioDataset(audio_list_val,target_list_val,n_audio_max,n_target_max)

batch_size_train = 32
batch_size_val = 64

train_loader = DataLoader(dataset_train, batch_size=batch_size_train,shuffle=True)
val_loader = DataLoader(dataset_val, batch_size=batch_size_val)

In [29]:
n_class = 79
model = Wav2Letter(n_class)
model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
criterion = torch.nn.CTCLoss(reduction='sum')

In [30]:
def train(model,train_loader,val_loader,optimizer,criterion,n_epoch) :
    
    optimal_model = copy.copy(model)
    av_lev_dist_old = 2
    
    for e in range(n_epoch) :
    
        total_training_loss = 0   
        model.train()
        for batch in train_loader :

            optimizer.zero_grad()   

            audio = batch[0]
            targets = batch[1]
            target_lengths = batch[2]        
            current_batch_size = audio.size()[0]
            output = model(audio)

            # this basically a tensor vector of the length the size of the current
            # batch size, each entry being the length of the predictions (determined in the model)
            input_lengths = torch.full(size=(current_batch_size,), fill_value=output.size()[-1], dtype=torch.long)

            # loss = ctc_loss(input, target, input_lengths, target_lengths)
            loss = criterion(output.transpose(1, 2).transpose(0, 1),targets,input_lengths,target_lengths)        
            total_training_loss += float(loss.cpu())

            loss.backward()
            optimizer.step()

        total_val_loss = 0  
        total_lev_dist = 0
        model.eval()      
        for batch in val_loader :

            audio = batch[0]
            targets = batch[1]
            target_lengths = batch[2]        
            current_batch_size = audio.size()[0]
            output = model(audio)        

            input_lengths = torch.full(size=(current_batch_size,), fill_value=output.size()[-1], dtype=torch.long)
            loss = criterion(output.transpose(1, 2).transpose(0, 1),targets,input_lengths,target_lengths)

            total_val_loss += float(loss.cpu())

            targets = targets.cpu().numpy().astype('int')     
            outmax = torch.argmax(output,dim=1).cpu().numpy()
            for i, vec in enumerate(outmax):

                original = postprocessor.target2kana(targets[i]) 
                predicted = postprocessor.target2kana(vec,refine = True)
                lev_dist = postprocessor.levenshtein(original,predicted)
                total_lev_dist += lev_dist/len(original)

        av_lev_dist = total_lev_dist/n_val

        if av_lev_dist < av_lev_dist_old :

            optimal_model = copy.copy(model)
            av_lev_dist_old = copy.copy(av_lev_dist)

        print(e,total_training_loss/n_train,total_val_loss/n_val,av_lev_dist)
        
    return optimal_model

In [31]:
n_epoch = 100
trained_model1 = train(model,train_loader,val_loader,optimizer,criterion,n_epoch)

0 56.669508341999794 45.14321802613731 0.985475792988314
1 16.980112121808506 14.974918473742044 1.0
2 14.451047081319212 13.090685301511634 1.0
3 11.748338184816719 11.076550873770739 0.9791875347801889
4 8.606692453925817 7.592670039461929 0.7012852108010715
5 6.063885069158833 5.251799059630635 0.4649925802263032
6 4.518263772599994 4.712215773848341 0.397881389617617
7 3.515137702503098 3.833461194683196 0.30757810636775595
8 2.890733004058668 3.3247794786558327 0.26352518748178183
9 2.432168199047308 2.9649124909720954 0.22419773696902245
10 2.1811498426108282 2.7348933912477826 0.22943463974348774
11 1.9528284670912932 3.118015575886569 0.2425159657630443
12 1.692712571138796 3.2082599518892163 0.2502590295995973
13 1.490934048159003 2.576142567426016 0.20490235047831035
14 1.347614708017551 3.077576435070006 0.22903251450830753
15 1.21317317835254 2.8965778764778864 0.2253345522961549
16 1.1686531679086207 2.8104662123028943 0.20754961973659794
17 1.1135071032564803 2.6144378865

In [22]:
def infer(model,val_loader) :

    total_lev_dist = 0
    n_val = len(val_loader.dataset)
    for data in val_loader :
        
        audio = data[0]
        targets = data[1].cpu().numpy().astype('int')     
        output = model(audio)
        outmax = torch.argmax(output,dim=1).cpu().numpy()
        
        for i, vec in enumerate(outmax):
            
            original = postprocessor.target2kana(targets[i]) 
            predicted = postprocessor.target2kana(vec,refine = True)
            lev_dist = postprocessor.levenshtein(original,predicted)
            total_lev_dist += lev_dist/len(original)
            
            print(original, predicted,lev_dist,round(lev_dist/len(original),2))
            
        av_lev = total_lev_dist/n_val
        
    print('average Levenshtein distance',av_lev)

In [23]:
infer(trained_model1,val_loader)

ギター ギター 0 0.0
スット スット 0 0.0
キョーチョー キョーチョー 0 0.0
クワシイ クワシイ 0 0.0
フトル フトール 1 0.33
タイダン カイダン 1 0.25
ウチアワセル ウチアワセル 0 0.0
ジンジャ ジンジ 1 0.25
シャチョー シャチョー 0 0.0
ツバ ツバ 0 0.0
ケンブツニン ケンレツン 2 0.33
マル マレル 1 0.5
ササヤク サヤク 1 0.25
シチ ヒチ 1 0.5
ミダレル ロレル 2 0.5
オサエル オサル 1 0.25
イチドニ イチド 1 0.25
ガクレキ タクレキ 1 0.25
テクビ テクビ 0 0.0
センロ センゴ 1 0.33
バッキン バッキン 0 0.0
カサイ カサイ 0 0.0
トシヨリ トシリ 1 0.25
ゴーイン ン 3 0.75
ウィスキー エスキイ 3 0.6
シュクダイ ヒクライ 3 0.6
ユノミ レノミ 1 0.33
オサマル オサマル 0 0.0
フラフラ フラクラ 1 0.25
ヒドイ ヒイ 1 0.33
モエル モル 1 0.33
ミダシ ミダシ 0 0.0
チョーキ チョーキ 0 0.0
ペース テイス 2 0.67
セイホーケイ セイホーケイ 0 0.0
ムシロ シロ 1 0.33
ツバメ ツダメ 1 0.33
リョード ヨード 2 0.5
エ テ 1 1.0
シンブン シン 2 0.5
シュショー ュショー 1 0.2
ノリ モイ 2 1.0
アタリ アタリ 0 0.0
ナガイキ ワナイキ 2 0.5
サカ サカ 0 0.0
ゲンロン ゲンヨン 1 0.25
コーバン コーバン 0 0.0
ヨーキ ヨーキ 0 0.0
ザイガク ザイノク 1 0.25
サク サコ 1 0.5
チキン チキン 0 0.0
キンイロ ヒュ 4 1.0
キセイ キセレイ 1 0.33
ボーガイ イ 3 0.75
トリシマリ トリスマリ 1 0.2
ウチコム ウチコン 1 0.25
シュッコク シュッコク 0 0.0
ヨリカカル ヨリカル 1 0.2
アカチャン アカチャウ 1 0.2
ダンセイ ダンセイ 0 0.0
サイショ サシ 2 0.5
ナガス マナス 2 0.67
カコム ハコ 2 0.67
ビン リン 1 0.5
カタナ ハタマ 2 0.67
ナ

In [21]:
model_aug = Wav2Letter(n_class)
model_aug = model_aug.cuda()
optimizer = torch.optim.Adam(model_aug.parameters(),lr=1e-4)
criterion = torch.nn.CTCLoss(reduction='sum')

dataset_train_aug = AudioDataset(audio_list_train,target_list_train,n_audio_max,n_target_max,random_pad = True,change_speed=True)
dataset_val_aug = AudioDataset(audio_list_val,target_list_val,n_audio_max,n_target_max)

train_loader_aug = DataLoader(dataset_train_aug, batch_size=batch_size_train,shuffle=True)
val_loader_aug = DataLoader(dataset_val_aug, batch_size=batch_size_val)

n_epoch = 10
trained_model2 = train(model_aug,train_loader_aug,val_loader_aug,optimizer,criterion,n_epoch)
infer(trained_model2,val_loader)

0 70.53784700015038 18.051293782280364 0.9929048414023373
1 16.164517313828053 15.455222510336238 1.0
2 14.909514104103556 13.880010436094663 1.0
3 12.689145412692776 11.022015961661364 1.0
4 10.305488668523163 8.64447041863392 0.8376553515117797
5 8.280627076214452 6.841288724208316 0.6723003948379581
6 6.7949861912205405 5.8177494564916135 0.5678637676550866


KeyboardInterrupt: 